In [1]:
! pip install tensorboardX pandas simpletransformers transformers

     |████████████████████████████████| 204kB 5.0MB/s 
     |████████████████████████████████| 153kB 16.1MB/s 
     |████████████████████████████████| 552kB 11.1MB/s 
     |████████████████████████████████| 3.7MB 29.4MB/s 
     |████████████████████████████████| 870kB 52.8MB/s 
     |████████████████████████████████| 1.0MB 36.2MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=fb00152aaf86487ffb7ff60d9b12fe1923799261d340a287e317bb7ad041f375
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=6c9559a7c33ccf152bb08214486e7cbd440d830f253f2792d098e328cfcf09e7
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built seqeval sacremoses


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [4]:
! pip install https://download.pytorch.org/whl/cu100/torch-1.4.0%2Bcu100-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 723.9MB 25kB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [5]:
! pip uninstall torchvision -y
! pip install torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html

Uninstalling torchvision-0.5.0:
  Successfully uninstalled torchvision-0.5.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 4.1MB 658kB/s 


In [6]:
! pip uninstall apex -y
! rm -rf apex
! git clone https://www.github.com/nvidia/apex
! cd apex && python setup.py install --cpp_ext --cuda_ext

Cloning into 'apex'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 6254 (delta 56), reused 53 (delta 34), pack-reused 6169
Receiving objects: 100% (6254/6254), 13.66 MiB | 20.24 MiB/s, done.
Resolving deltas: 100% (4106/4106), done.
torch.__version__  =  1.4.0+cu100
setup.py:46: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
  warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
wr

In [0]:
! cp ./drive/My\ Drive/mlin/hindicorpus/hi_3500.csv ./

In [1]:
import pandas as pd 

df = pd.read_csv("./hi_3500.csv", header=0, names=['review', 'sentiment'])
print(df.head())

                                              review sentiment
0  गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...  negative
1  ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है  negative
2  यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...  negative
3  अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...  negative
4                    कश्मीर में हो रहा है जल जिहाद ।  negative


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], random_state=91)

In [3]:
print(set(y_train.values))

{'neutral', 'negative', 'positive'}


In [4]:
from sklearn import preprocessing
labeler = preprocessing.LabelEncoder()

df2 = pd.DataFrame(X_train)
df2['label'] = labeler.fit_transform(y_train)
print(df2.head())

                                                 review  label
2378  RT @itsparvezsagar: शैलजा मर्डरः मृतका का पोस्...      1
1450  सेट और कॉस्ट्यूम में कल्पना और बजट की कटौती से...      0
5513  स प्रदर्शन से मुझे और परिपक्व गेंदबाज बनने में...      2
3061  RT @neerajjournalis: वोट के तैयार है काशी और ह...      1
5975                           मेरी यात्रा आनंदप्रद रही      2


In [26]:
from simpletransformers.classification import ClassificationModel

# set use_cuda=False on CPU-only platforms
model = ClassificationModel('bert', 'bert-base-multilingual-uncased', num_labels=3, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 3
})
model.train_model(df2)
print("trained model")

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:249: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.102954

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:113: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 1.090263Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.994698

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.510946Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.929150Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.521209Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.492807


Running loss: 0.102267


Running loss: 0.242605

trained model


In [29]:
eval_frame = pd.DataFrame(X_test)
eval_frame['label'] = labeler.fit_transform(y_test)
print(eval_frame.head())

                                                 review  label
8596  RT @aajtak: #HallaBol\n@sambitswaraj बीजेपी की...      1
8098  RT @aajtak: पीएम मोदी ने बताई सर्जिकल स्ट्राइक...      1
117   ~चप्पड़गंजू नसीहत दे रहा है की गुरु का सम्मान ...      0
7407                             यह वस्त्र परिधानीय है"      0
4836  इस पर चुप्पी तोड़ते हुए रामपाल ने कहा है कि उन...      2


In [6]:
print(len(eval_frame))

2269


In [28]:
result, model_outputs, wrong_predictions = model.eval_model(eval_frame)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions (by input I think?):")
print(bads)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:587: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



wrong predictions (by input I think?):
{0: 208, 2: 208, 1: 36}


In [30]:
(1 - (208 + 208 + 36) / 2269) * 100

80.07933010136624

80.1% Accuracy, 4 epochs, SimpleTransformers + Multilingual BERT

In [11]:
! pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 37kB/s 
     |████████████████████████████████| 512kB 40.0MB/s 
     |████████████████████████████████| 3.8MB 44.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=51fc313adb53e4bda760e1ed2933f1269a1bf300d8d58a3a9172c15b1f4afa44
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc

In [13]:
! rm -rf electra
! git clone https://github.com/MonsoonNLP/electra

Cloning into 'electra'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 95 (delta 6), reused 11 (delta 4), pack-reused 81
Unpacking objects: 100% (95/95), done.


In [0]:
! mkdir ./drive/My\ Drive/mlin/hindicorpus/trainer/finetuning_data/movies

In [0]:
df2.to_csv("./drive/My Drive/mlin/hindicorpus/trainer/finetuning_data/movies/train.tsv", sep="\t")
eval_frame.to_csv("./drive/My Drive/mlin/hindicorpus/trainer/finetuning_data/movies/dev.tsv", sep="\t")

In [31]:
! cd electra && python run_finetuning.py --data-dir ../drive/My\ Drive/mlin/hindicorpus/trainer --model-name babylil --hparams '{"task_names": ["movies"], "num_train_epochs": 3}'

Config: model=babylil, trial 1/1
answerable_classifier True
answerable_uses_start_logits True
answerable_weight 0.5
beam_size 20
data_dir ../drive/My Drive/mlin/hindicorpus/trainer
debug False
do_eval True
do_lower_case True
do_train True
doc_stride 128
double_unordered True
embedding_size 128
eval_batch_size 32
gcp_project None
init_checkpoint ../drive/My Drive/mlin/hindicorpus/trainer/models/babylil
iterations_per_loop 1000
joint_prediction True
keep_all_models True
layerwise_lr_decay 0.8
learning_rate 0.0001
log_examples False
max_answer_length 30
max_query_length 64
max_seq_length 128
model_dir ../drive/My Drive/mlin/hindicorpus/trainer/models/babylil/finetuning_models/movies_model
model_hparam_overrides {}
model_name babylil
model_size small
n_best_size 20
n_writes_test 5
num_tpu_cores 1
num_train_epochs 3
num_trials 1
predict_batch_size 32
preprocessed_data_dir ../drive/My Drive/mlin/hindicorpus/trainer/models/babylil/finetuning_tfrecords/movies_tfrecords
qa_eval_file <built-in m